# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

Analysis
1. Most of the hotels within the ideal parameters are close to the equator.
2. There is only one hotel in the United States that met all the criteria.
3. There were no hotel matches in Europe, Australia, or China, amongst other countries, based on the selected criteria.


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint


# Import API key
from api_keys import g_key
from api_keys import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#initial dataframe setup
df = pd.read_csv(os.path.join("..", "output_data", "cities.csv"))
df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)


In [4]:
# Store 'Lat' and 'Lng' into a variable called 'locations'
locations = df[["Lat", "Lng"]]

# Store humidity into a variable called 'weight'
weight = df["Humidity"].astype(float)


In [5]:
# Create the mapping figure
fig = gmaps.figure()

# Create the heatmap layer using locations and humidity
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=0.9)
# Add heatmap layer to figure
fig.add_layer(heat_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
##ideal weather conditions is 80 to 95 degrees, max wind less than 20, humidity 70-90%

# use iterrows to iterate through pandas DataFrame
for index, row in df.iterrows():
    #second URL complete
    base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
    
    # get city from DataFrame
    city_type = row['City']

    # add a new key to our params dictionary that will store our keyword to search for
    #params['keyword'] = city_type

    # assemble url and make API request
    print(f"Retrieving Results for : {city_type}.")
    response = requests.get(base_url + "&q="+ city_type).json()
    pprint(response)
    break
    # Use exception handling in case any of our city attributes are missing
    # NOTE: if our code hits a KeyError or IndexError on ANY of the lines of code below, it will skip any unexecuted code and move to the next iteration
    try:
    # extract results
        main= response["main"]
        wind = response["wind"]

        windspeed = wind["speed"]
        humidity = main["humidity"]
        temp_max = main["temp_max"]

        print(f"{city_type} has humidity: {humidity}, windspeed: {windspeed}, temp max: {temp_max}")
        
        #need to add conditionals based on criteria selected
        df.loc[index, 'Humidity'] = humidity
        df.loc[index, 'Max Temp'] = temp_max
        df.loc[index, 'Wind Speed'] = windspeed
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

new_df = df[(90>=df['Humidity']) & (df['Humidity']>=70)]
new_df = new_df[(95>=new_df['Max Temp']) & (new_df['Max Temp']>=80)]
new_df = new_df[20>=new_df['Wind Speed']]


#alternative combined conditional test    
#new_df = df[(90<=df['Humidity']>=70)&(95<=df['Max Temp']>=80)&(20<=df['Wind Speed'])]



Retrieving Results for : ostrovnoy.
{'base': 'stations',
 'clouds': {'all': 100},
 'cod': 200,
 'coord': {'lat': 68.05, 'lon': 39.51},
 'dt': 1600208385,
 'id': 556268,
 'main': {'feels_like': 35.78,
          'grnd_level': 1015,
          'humidity': 79,
          'pressure': 1017,
          'sea_level': 1017,
          'temp': 44.22,
          'temp_max': 44.22,
          'temp_min': 44.22},
 'name': 'Ostrovnoy',
 'rain': {'1h': 0.11},
 'sys': {'country': 'RU', 'sunrise': 1600224175, 'sunset': 1600271447},
 'timezone': 10800,
 'visibility': 10000,
 'weather': [{'description': 'light rain',
              'icon': '10n',
              'id': 500,
              'main': 'Rain'}],
 'wind': {'deg': 330, 'speed': 10.42}}


In [10]:
#data frame that extracts cities that meet conditionals
new_df = df[(90>=df['Humidity']) & (df['Humidity']>=70)]
new_df.sort_values('Max Temp', ascending=False).head(10)


new_df = new_df[(95>=new_df['Max Temp']) & (new_df['Max Temp']>=80)]
new_df

new_df = new_df[20>=new_df['Wind Speed']]
new_df

#drop any null values
final_df= new_df.dropna()

#final_df.shape

#display final data frame
final_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
5,5,atuona,78,PF,1558378755,76,-9.80,-139.03,81.96,16.49
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23
45,45,rosita,72,NI,1558378762,71,13.92,-84.40,84.12,4.41
49,49,garchitorena,59,PH,1558378763,75,13.88,123.70,84.30,6.11
54,54,tiarei,20,PF,1558378764,70,-17.53,-149.33,84.20,10.29
72,72,butaritari,55,KI,1558378767,77,3.07,172.79,83.94,9.13
74,74,hobyo,91,SO,1558378767,71,5.35,48.53,83.58,13.49
89,89,ternate,20,PH,1558378770,74,14.29,120.72,86.00,1.12
103,103,kapaa,20,US,1558378773,74,22.08,-159.32,80.60,10.29
108,108,bonthe,63,SL,1558378774,70,7.53,-12.50,85.38,9.78


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
#create new variable to store hotel names and hotel count
hotel_df = final_df

hotel_df["Hotel Name"]=np.nan

#hotel_df = hotel_df["hotel Count"]

# Normalize the hotel counts from 0 to 1 and store that in a new variable
#norm_count = (hotel_count - min(hotel_count)) / (max(hotel_count) - min(hotel_count))

# view the distribution of normalized hotel counts
#norm_count.hist()

# distance, IN METERS, within which the place results must live
target_radius = 5000
# keyword term to be matched against all of Google's indexed content
target_type = "lodging"
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# type of establishment that we'd like to use to filter our place results
target_keyword = "hotel"

for index, row in hotel_df.iterrows():
# geocoordinates for the hotel based on search
    #get more specific lat and lng coordinates
    target_location = str(row["Lat"])+","+str(row["Lng"])
    city=row['City']
# set up a dictionary to store all query parameters
    params = {
        "location": target_location,
        "keyword": target_keyword,
        "radius": target_radius,
    #     "rankby": "distance",
        "type": target_type,
        "key": g_key
    }

# run a request using our params dictionary
    response = requests.get(base_url, params=params)

# print the response url, avoid doing for public github repos in order to avoid exposing key
   # print(response.url)
    print(f'Getting hotels for {city}') 
# convert the response to json
    try:
        response_json = response.json()
        results = response_json['results']
        name = results[0]['name']
        lat = results[0]['geometry']['location']['lat']
        lng = results[0]['geometry']['location']['lng']

        hotel_df.loc[index,"Hotel Name"]= name
        hotel_df.loc[index,"Lat"]= lat
        hotel_df.loc[index,"Lng"]= lng
    except (KeyError, IndexError):
        print('Failed, skipping')
  



Getting hotels for atuona
Getting hotels for teknaf
Getting hotels for rosita
Getting hotels for garchitorena
Failed, skipping
Getting hotels for tiarei
Getting hotels for butaritari
Failed, skipping
Getting hotels for hobyo
Getting hotels for ternate
Failed, skipping
Getting hotels for kapaa
Getting hotels for bonthe
Getting hotels for port blair
Getting hotels for thinadhoo
Failed, skipping
Getting hotels for faanui
Getting hotels for arraial do cabo
Getting hotels for georgetown
Getting hotels for nagua
Getting hotels for sinnamary
Getting hotels for trat
Getting hotels for antsohihy
Getting hotels for kavieng
Getting hotels for monrovia
Getting hotels for hithadhoo
Getting hotels for salalah
Getting hotels for san juan del sur
Getting hotels for bandarbeyla
Getting hotels for tagusao
Failed, skipping
Getting hotels for port-gentil
Getting hotels for bodden town
Getting hotels for vila velha
Getting hotels for soyo
Getting hotels for cayenne
Getting hotels for san andres
Getting hot

In [16]:
#display new data frame with hotel names from previous loop
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
5,5,atuona,78,PF,1558378755,76,-9.807877,-139.030335,81.96,16.49,Simplicité Marquises
20,20,teknaf,1,BD,1558378758,85,20.865881,92.298120,84.66,8.23,Milky Resort
45,45,rosita,72,NI,1558378762,71,13.924066,-84.396941,84.12,4.41,Hotel Terciopelo
49,49,garchitorena,59,PH,1558378763,75,13.880000,123.700000,84.30,6.11,NaN
54,54,tiarei,20,PF,1558378764,70,-17.539562,-149.366494,84.20,10.29,Le Rocher de Tahiti
72,72,butaritari,55,KI,1558378767,77,3.070000,172.790000,83.94,9.13,NaN
74,74,hobyo,91,SO,1558378767,71,5.356321,48.534480,83.58,13.49,Osmani Hotel
89,89,ternate,20,PH,1558378770,74,14.290000,120.720000,86.00,1.12,NaN
103,103,kapaa,20,US,1558378773,74,22.042287,-159.337013,80.60,10.29,Hilton Garden Inn Kauai Wailua Bay
108,108,bonthe,63,SL,1558378774,70,7.525635,-12.500587,85.38,9.78,Bonthe Holiday Village


In [17]:
#loop to store hotel names and locations that will grab google lat and long instead of original amounts
#from source data that was just 2 decimal points
hotel_info = []
locations = []
for index, row in hotel_df.dropna().iterrows():
    hotel_info.append(row['Hotel Name'] + ', '+row['City'] +', '+row['Country'])
    locations.append((row['Lat'], row['Lng']))


# Create the heatmap layer using locations and hotels
hotel_layer = gmaps.symbol_layer(
    locations, 
    fill_color="green",
    stroke_color='navy',
    #fill_opacity=hotel_count,
    #stroke_opacity=hotel_count,
    # the below line creates a list of strings to populate the info-box when you click on a datapoint on the map
    info_box_content=hotel_info
)

# Create the mapping figure
fig = gmaps.figure()


# Add heatmap layer to figure
fig.add_layer(hotel_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

#add heat layer map and then hotel map on top
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))